# Dropbox Download Notebook
This notebook downloads the NMEA data from Dropbox, parses the multiple hourly files into a single daily file, and saves it in the correct folder.

<b>Note:</b> This notebook requires the credentials file generated in the `dropbox_credentials.ipynb` notebook to be present in the same directory.

In [2]:
from dropbox import Dropbox
import json

The below code will ensure that the access token is valid and refresh it if necessary.

In [8]:
app_key = 'your_app_key_here'
app_secret = 'your_app_secret_here'

def load_credentials():
    try:
        with open('dropbox_creds.json', 'r') as f:
            creds = json.load(f)
            return creds
    except FileNotFoundError:
        print("Credentials file not found. Please run the dropbox_credentials.ipynb notebook first.")
        return None


def create_dropbox_client():
    creds = load_credentials()
    if creds is None:
        return None
    access_token = creds['access_token']
    refresh_token = creds['refresh_token']
    dbx = Dropbox(app_key=app_key, app_secret=app_secret, oauth2_access_token=access_token, oauth2_refresh_token=refresh_token)
    dbx.refresh_access_token()

    return dbx

dbx = create_dropbox_client()
if dbx is None:
    print("Failed to create Dropbox client. Exiting.")

print(dbx.users_get_current_account())

FileNotFoundError: [Errno 2] No such file or directory: 'dropbox_creds.json'